<a href="https://colab.research.google.com/github/pachterlab/seqspec/blob/devel/docs/UNIFORM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
title: Using seqspec
date: 2024-09-07
authors:
  - name: A. Sina Booeshaghi
---

`seqspec` enables uniform preprocessing of sequencing reads.

# Single-cell preprocessing
Single-cell data preprocessing is the procedure where

1. Sequencing reads are aligned to a reference
2. Barcodes errors are corrected
3. UMIs/reads are counted

The goal is to produce a count matrix, where rows are cells or samples and columns are biological features such as genes, proteins, or genomic regions.

There are many tools that perform single-cell RNA-sequencing preprocessing. For this tutorial we will use `kb-python` (which uses `kallisto` and `bustools`), `STARsolo`, `simpleaf` with `seqspec` to perform alignment and quantification. `kb_python` uses `kallisto` to perform read alignment and `bustools` to perform and barcode correction and UMI counting. `STARsolo` performs performs whole genome alignment and barcode error correction. Like `kb-python`, `simpleaf` uses two separate tools under the hood: `salmon` to perform read alignment and `alevin-fry` to perform barcode error correction and UMI counting.

Throughout this tutorial we will use the `dogmaseq-dig` dataset which is a multimodal assay (RNA/ATAC/PROTEIN/TAG). The `seqspec` for this dataset can be found here


## Install tools

To understand how each tool works, please review their code and manuscript:

| Tool | Code link | Manuscript link | Purpose |
|------|-----------|-----------------|---------|
| seqspec | [GitHub](https://github.com/pachterlab/seqspec) | [doi](https://doi.org/10.1093/bioinformatics/btae168) | Identify and extract elements in reads |
| kb-python | [GitHub](https://github.com/pachterlab/kb_python) | [doi](https://doi.org/10.1101/2023.11.21.568164) | Perform read alignment, error correction, and counting |
| gget | [GitHub](https://github.com/pachterlab/gget) | [doi](https://doi.org/10.1093/bioinformatics/btac836) | Fetch species-specific references |
| kallisto | [GitHub](https://github.com/pachterlab/kallisto) | [doi](https://doi.org/10.1038/nbt.3519) | Perform read alignment (used in kb-python) |
| bustools | [GitHub](https://github.com/BUStools/bustools) | [doi](https://doi.org/10.1038/s41587-021-00870-2) | Perform barcode error correction and UMI counting (used in kb-python) |
| BUS file | [GitHub](https://github.com/BUStools/BUS-format) | [doi](https://doi.org/10.1093/bioinformatics/btz279) | Store barocdes, umis, and read alignments (used in kb-python) |
| STARsolo | [GitHub](https://github.com/alexdobin/STAR/) | [doi](https://doi.org/10.1101/2021.05.05.442755) | Perform read alignment, error correction, and counting |
| simpleaf | [GitHub](https://github.com/COMBINE-lab/simpleaf) | [doi](https://doi.org/10.1093/bioinformatics/btad614) | Perform read alignment, error correction, and counting |

In [2]:
# Install kb-python, seqspec, gget
! pip install --quiet kb-python gget > /dev/null 2>&1  # installing kb-python autoinstalls kallisto and bustools
! pip install --quiet git+https://github.com/pachterlab/seqspec@devel > /dev/null 2>&1

# Verify installations
! seqspec --version
! kb --version
! gget --version

# Install STARsolo and verify installation
! wget --quiet --show-progress https://github.com/alexdobin/STAR/archive/2.7.11b.tar.gz
! tar -xzf 2.7.11b.tar.gz > /dev/null 2>&1
! mv /content/STAR-2.7.11b/bin/Linux_x86_64/STAR /usr/bin
! STAR --version

# Install alevin-fry, simpleaf and verify installation
! curl -S --proto '=https' --tlsv1.2 -LsSf https://github.com/COMBINE-lab/alevin-fry/releases/download/v0.10.0/alevin-fry-installer.sh | sh > /dev/null 2>&1
! $HOME/.cargo/bin/alevin-fry --version

! curl -S --proto '=https' --tlsv1.2 -LsSf https://github.com/COMBINE-lab/simpleaf/releases/download/v0.17.2/simpleaf-installer.sh | sh > /dev/null 2>&1
%env ALEVIN_FRY_HOME="$HOME/.cargo/bin/alevin-fry"
! $HOME/.cargo/bin/simpleaf --version

!wget --quiet --show-progress https://github.com/jqlang/jq/releases/download/jq-1.7.1/jq-linux-amd64
!chmod +x jq-linux-amd64
!mv jq-linux-amd64 /usr/bin/jq

seqspec 0.2.0
usage: kb [-h] [--list] <CMD> ...

kb_python 0.28.2

positional arguments:
  <CMD>
    info      Display package and citation information
    compile   Compile `kallisto` and `bustools` binaries from source
    ref       Build a kallisto index and transcript-to-gene mapping
    count     Generate count matrices from a set of single-cell FASTQ files

options:
  -h, --help  Show this help message and exit
  --list      Display list of supported single-cell technologies
gget version: 0.28.6
2.7.11b.tar.gz          [        <=>         ]  11.89M  7.50MB/s    in 1.6s    
2.7.11b
alevin-fry 0.10.0
env: ALEVIN_FRY_HOME="$HOME/.cargo/bin/alevin-fry"
simpleaf 0.17.2
jq-linux-amd64      100%[===================>]   2.21M  --.-KB/s    in 0.02s   


## Download `seqspec` for the `dogmaseq-dig` data

In [3]:
! wget --quiet --show-progress https://raw.githubusercontent.com/pachterlab/seqspec/devel/examples/specs/dogmaseq-dig/spec.yaml

spec.yaml           100%[===================>]  14.09K  --.-KB/s    in 0s      


In [4]:
! seqspec print spec.yaml

                                                                  ┌─'ghost_protein_truseq_read1:0'
                                                                  ├─'protein_truseq_read1:33'
                                                                  ├─'protein_cell_bc:16'
                                 ┌─protein────────────────────────┤
                                 │                                ├─'protein_umi:12'
                                 │                                ├─'protein_seq:15'
                                 │                                └─'protein_truseq_read2:34'
                                 │                                ┌─'tag_truseq_read1:33'
                                 │                                ├─'tag_cell_bc:16'
                                 ├─tag────────────────────────────┼─'tag_umi:12'
                                 │                                ├─'tag_seq:15'
─────────────────────────────────┤               

In [12]:
! seqspec file -m rna -f json -s file -k all spec.yaml

[
    {
        "file_id": "rna_R1_SRR18677638.fastq.gz",
        "filename": "rna_R1_SRR18677638.fastq.gz",
        "filetype": "fastq",
        "filesize": 18499436,
        "url": "https://github.com/pachterlab/seqspec/raw/devel/examples/specs/dogmaseq-dig/fastqs/rna_R1_SRR18677638.fastq.gz",
        "urltype": "https",
        "md5": "7eb15a70da9b729b5a87e30b6596b641"
    },
    {
        "file_id": "rna_R2_SRR18677638.fastq.gz",
        "filename": "rna_R2_SRR18677638.fastq.gz",
        "filetype": "fastq",
        "filesize": 45812569,
        "url": "https://github.com/pachterlab/seqspec/raw/devel/examples/specs/dogmaseq-dig/fastqs/rna_R2_SRR18677638.fastq.gz",
        "urltype": "https",
        "md5": "5e6915770e50f72e462e5b2575089c66"
    },
    {
        "file_id": "RNA-737K-arc-v1.txt",
        "filename": "RNA-737K-arc-v1.txt",
        "filetype": "txt",
        "filesize": 2142553,
        "url": "https://github.com/pachterlab/qcbc/raw/main/tests/10xMOME/RNA-737K-arc-v1.t

In [13]:
! seqspec file -m rna -f json -s file -k all spec.yaml     | jq '.[].url' | xargs wget --continue --quiet --show-progress
! seqspec file -m atac -f json -s file -k all spec.yaml    | jq '.[].url' | xargs wget --continue --quiet --show-progress
! seqspec file -m tag -f json -s file -k all spec.yaml     | jq '.[].url' | xargs wget --continue --quiet --show-progress
! seqspec file -m protein -f json -s file -k all spec.yaml | jq '.[].url' | xargs wget --continue --quiet --show-progress

rna_R1_SRR18677638. 100%[===================>]  17.64M  --.-KB/s    in 0.07s   
rna_R2_SRR18677638. 100%[===================>]  43.69M   270MB/s    in 0.2s    
RNA-737K-arc-v1.txt 100%[===================>]   2.04M  --.-KB/s    in 0.03s   
atac_R1_SRR18677642 100%[===================>]  38.33M   165MB/s    in 0.2s    
atac_R2_SRR18677642 100%[===================>]  20.01M  --.-KB/s    in 0.1s    
atac_R3_SRR18677642 100%[===================>]  34.88M  --.-KB/s    in 0.1s    
ATA-737K-arc-v1.txt 100%[===================>]   2.35M  --.-KB/s    in 0.03s   
tag_R1_SRR18677640. 100%[===================>]  17.20M  --.-KB/s    in 0.07s   
tag_R2_SRR18677640. 100%[===================>]   7.13M  --.-KB/s    in 0.05s   
RNA-737K-arc-v1.txt 100%[===================>]   2.04M  --.-KB/s    in 0.03s   
tag_feature_barcode 100%[===================>]     208  --.-KB/s    in 0s      
protein_R1_SRR18677 100%[===================>]  17.33M  --.-KB/s    in 0.1s    
protein_R2_SRR18677 100%[===============

In [14]:
! gunzip *.txt.gz

## Single-cell/nuclei RNAseq quantification

### `kb-python (kallisto bustools)`

In [31]:
# seqspec commands to get onlist, technology string, and file
! seqspec index -m rna -t kb -s file spec.yaml
! seqspec file -m rna -s region -k filename spec.yaml
! seqspec file -m rna -s read -f paired -k filename spec.yaml  | tr "\t\n" "  "

0,0,16:0,16,28:1,0,102
RNA-737K-arc-v1.txt
rna_R1_SRR18677638.fastq.gz rna_R2_SRR18677638.fastq.gz 

In [ ]:
# standard reference
! kb ref \
-i index.idx \
-g t2g.txt \
-f1 transcriptome.fa \
$(gget ref --ftp -w dna,gtf homo_sapiens)

# standard quantification
! kb count \
--h5ad \
-t 16 \
-m 32G \
-i index.idx \
-g t2g.txt \
-o kb_out \
-x $(seqspec index -m rna -t kb -s file spec.yaml) \
-w $(seqspec file -m rna -s region -k filename spec.yaml) \
$(seqspec file -m rna -s read -f paired -k filename spec.yaml  | tr "\t\n" "  ")

In [ ]:
# spliced, unspliced, ambiguous reference
! kb ref \
--workflow nac \
-i index.idx \
-g t2g.txt \
-f1 spl.fa \
-f2 unspl.fa \
-c1 spl.t2c.txt \
-c2 unspl.t2c.txt \
$(gget ref --ftp -w dna,gtf homo_sapiens)

# spliced, unspliced, ambiguous quantification
! kb count \
--h5ad \
--workflow=nac \
-t 32 \
-m 64G \
-i index.idx \
-g t2g.txt \
-c1 spl.t2c.txt \
-c2 unspl.t2c.txt \
-o kb_out_nac \
-x $(seqspec index -m rna -t kb -s file spec.yaml) \
-w $(seqspec file -m rna -s region -k filename spec.yaml) \
$(seqspec file -m rna -s read -f paired -k filename spec.yaml  | tr "\t\n" "  ")

### `STARsolo`

In [ ]:
# download reference
## todo

# run quantification
! star \
--soloFeatures Gene \
--genomeDir index \
--soloType Droplet \
--soloCBwhitelist \
$(seqspec file -m rna -s region -k filename spec.yaml) \
$(seqspec index -m rna -t starsolo -s file spec.yaml) \
--readFilesIn $(seqspec file -m rna -s read -f paired -k filename spec.yaml  | tr "\t\n" "  ")

In [33]:
! seqspec file -m rna -s read -f paired -k filename spec.yaml | awk '{print "-1 "$1" -2 "$2}'

-1 rna_R1_SRR18677638.fastq.gz -2 rna_R2_SRR18677638.fastq.gz


### `simpleaf`

In [ ]:
! mkdir -p simpleaf_ref

# Download reference genome and gene annotations
! wget -qO- https://cf.10xgenomics.com/supp/cell-exp/refdata-gex-GRCh38-2020-A.tar.gz | tar xzf - --strip-components=1 -C ./simpleaf_ref

# simpleaf index
! simpleaf index \
--output ./out \
--fasta ./simpleaf_ref/fasta/genome.fa \
--gtf ./simpleaf_ref/genes/genes.gtf \
--rlen 91 \
--threads 16 \
--use-piscem  # remove this if missing piscem

! simpleaf quant \
-r cr-like \
-i simpleaf_ref/ \
-m t2g.txt \
-c $(seqspec index -m rna -t simpleaf -s file spec.yaml) \
-o out/ -x $w \
$(seqspec file -m rna -s read -f paired -k filename spec.yaml | awk '{print "-1 "$1" -2 "$2}')

## Single-cell/nuclei TAG quantification

### `kb-python (kallisto bustools)`

In [ ]:
# build alignment reference
kb ref \
--workflow kite \
-i index.idx \
-g t2g.txt \
-f1 transcriptome.fa \
tag_feature_barcodes.txt

w=$(seqspec onlist -m tag -o onlist.txt -s region-type -i barcode spec.yaml)
x=$(seqspec index -m tag -t kb -s file spec.yaml)
f=$(seqspec file -m tag -s read -f paired -k url spec.yaml  | tr "\t\n" "  ")

# perform alignment, error correction, and counting
kb count \
--workflow kite \
-i index.idx \
-g t2g.txt \
-x $x \
-w $w \
-o out --h5ad -t 2 \
$f

## Single-cell/nuclei PROTEIN quantification

### `kb-python` (kallisto bustools)

In [ ]:
# build alignment reference
kb ref \
--workflow kite \
-i index.idx \
-g t2g.txt \
-f1 transcriptome.fa \
protein_feature_barcodes.txt

w=$(seqspec onlist -m protein -o onlist.txt -s region-type -i barcode spec.yaml)
x=$(seqspec index -m protein -t kb -s file spec.yaml)
f=$(seqspec file -m protein -s read -f paired -k url spec.yaml  | tr "\t\n" "  ")

# perform alignment, error correction, and counting
kb count \
--workflow kite \
-i index.idx \
-g t2g.txt \
-x $x \
-w $w \
-o out --h5ad -t 2 \
$f

## Single-cell/nuclei CRISPR quantification

Note that single-cell CRISPR guide RNAs can be quantified in the same way as TAG and PROTEIN data. Simply supply the guide RNA barcode file as the “feature barcodes” file.

## Single-cell/nuclei ATAC quantification